In [ ]:
"""
    Instruções:
        1- Download dos dados municipais do estado de Minas Gerais;
        2- Transformação dos dados de desmatamento;
        3- Processamento dos dados:
            -reprojeção para EPSG:31983;
            -cálculo de área;
        4- Geração dos arquivos de saída no formato GeoJSON;
"""

In [38]:
import geopandas as gpd
import requests

def main():
    # URL do dataset
    url = "https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json"

    # Baixar o dataset
    response = requests.get(url)
    data = response.json()

    # Carregar em um GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(data)

    # Definir o CRS original (WGS 84)
    gdf.set_crs(epsg=4326, inplace=True)

    # Transformar para EPSG:31983
    gdf = gdf.to_crs(epsg=31983)

    # Calcular a área em km² e adicionar como uma nova coluna
    gdf['area_km2'] = gdf['geometry'].area / 10**6

    # Salvar em um arquivo GeoJSON
    gdf.to_file('municipios-mg.geojson', driver='GeoJSON')

if __name__ == "__main__":
    main()

In [47]:
### extract : https://cidades.ibge.gov.br/brasil/sintese/mg?indicadores=96385,47001

import pandas as pd

# Carregar o CSV
caminho_arquivo = "populacao-pib-municipios-mg.csv"
df = pd.read_csv(caminho_arquivo)

# Função para limpar os valores monetários e de população
def limpar_valor_monetario(valor):
    return valor.replace('R$', '').replace('.', '').replace(',', '.').strip()

def limpar_valor_populacao(valor):
    return valor.replace(' pessoas', '').replace('.', '').strip()

# Limpar os dados nas colunas especificadas
df['PIB per capita'] = df['PIB per capita'].apply(limpar_valor_monetario)
df['População'] = df['População'].apply(limpar_valor_populacao)

# Salvar o arquivo CSV limpo
caminho_arquivo_saida = "dados/pib-municipios-mg_limpo.csv"
df.to_csv(caminho_arquivo_saida, index=False)

df



,Municípios,Gentílico,PIB per capita,População
0,Abadia dos Dourados,mineiro,25174.26,6272
1,Abaeté,mineiro,23266.1,22675
2,Abre Campo,mineiro,19553.86,13927
3,Acaiaca,mineiro,16247.06,3909
4,Açucena,mineiro,17888.2,8943
...,...,...,...,...
848,Virginópolis,mineiro,17587.98,10314
849,Virgolândia,mineiro,11818.51,4552
850,Visconde do Rio Branco,mineiro,31179.87,39160
851,Volta Grande,mineiro,26633.91,4443


In [49]:
### Transform ###

import geopandas as gpd
import pandas as pd

# Caminhos dos arquivos
geojson_path = 'dados/municipios-mg.geojson'
csv_path = 'dados/pib-municipios-mg_limpo.csv'

# Carregar o arquivo GeoJSON
geo_df = gpd.read_file(geojson_path)

# Carregar o arquivo CSV
csv_df = pd.read_csv(csv_path)

# Renomear a coluna 'name' para 'Municípios' no DataFrame GeoJSON para consistência
geo_df.rename(columns={'name': 'Municípios'}, inplace=True)

# Mesclar csv_df com geo_df nas colunas 'Municípios', 'geometry' e 'area_km2'
merged_df = pd.merge(csv_df, geo_df[['Municípios', 'geometry', 'area_km2']], on='Municípios', how='left')

# Remover linhas com valores nulos na coluna 'geometry'
merged_df = merged_df[merged_df['geometry'].notna()]

# Definir 'Municípios' como o índice do DataFrame mesclado
merged_df = merged_df.set_index('Municípios')

# Salvar o DataFrame mesclado em um novo arquivo CSV
output_csv_path = 'dados/pib_municipios_with_geo.csv'
merged_df.to_csv(output_csv_path)

# Exibir o DataFrame resultante
merged_df



,Gentílico,PIB per capita,População,geometry,area_km2
Municípios,,,,,
Abadia dos Dourados,mineiro,25174.26,6272,"POLYGON ((241429.802 7990562.876, 242359.697 7...",895.115178
Abaeté,mineiro,23266.10,22675,"POLYGON ((480831.209 7911603.048, 481764.623 7...",1814.504046
Abre Campo,mineiro,19553.86,13927,"POLYGON ((775002.321 7771717.815, 776244.590 7...",472.049598
Acaiaca,mineiro,16247.06,3909,"POLYGON ((696872.185 7747792.052, 697305.640 7...",100.723996
Açucena,mineiro,17888.20,8943,"POLYGON ((775332.208 7905556.756, 775804.655 7...",812.875627
...,...,...,...,...,...
Virginópolis,mineiro,17587.98,10314,"POLYGON ((741901.325 7935223.813, 744154.104 7...",441.420798
Virgolândia,mineiro,11818.51,4552,"POLYGON ((783014.609 7968328.380, 782934.821 7...",281.912823
Visconde do Rio Branco,mineiro,31179.87,39160,"POLYGON ((719981.237 7683148.509, 720339.567 7...",241.559292


In [4]:
## Concatenar arquivos de desmatamento ##

import geopandas as gpd
import pandas as pd

def merge_and_save_geodataframes(file_path1, file_path2, output_file_path):
    # Ler os arquivos GeoPackage
    gdf1 = gpd.read_file(file_path1)
    gdf2 = gpd.read_file(file_path2)

    # Combinar os dois GeoDataFrames
    combined_gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))

    # Transformar para a projeção EPSG:31983
    combined_gdf = combined_gdf.to_crs(epsg=31983)

    # Salvar em um arquivo GeoJSON
    combined_gdf.to_file(output_file_path, driver='GeoJSON')

# Caminhos para os arquivos GeoPackage
file_path1 = 'dados/desmatamento_ago22.gpkg'
file_path2 = 'dados/desmatamento_set22.gpkg'
output_file_path = 'dados/focos-desmatamento-mg.geojson'

# Executar a função
merge_and_save_geodataframes(file_path1, file_path2, output_file_path)
